## Agile Hardware Design
***
## Functional Programming: Reduce

<img src="./images/chisel_logo.svg" alt="agile hardware design logo" style="width:20%;float:right"/>

- Modified by Peter Hanpin Chen based on

1. UC Berkeley Bootcamp with scala configuration (load-ivy.sc)

- https://github.com/freechipsproject/chisel-bootcamp

2. Prof. Scott Beamer,sbeamer@ucsc.edu

- [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)

## Plan for Today

* reduce + fold
* Scala type signatures
* zipWithIndex
* FP considerations

## Loading The Chisel Library Into a Notebook

In [1]:
// interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/011_FP_Reduce/001_Code/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Motivation for `reduce` and `fold`

* In the last lecture, we applied a function to each element (e.g. `map`, `foreach`, `zip`)
  * Resulting collection (if there is one), has same size as input collection

* What about if we want to combine things (collapse)?

* How do we gracefully handle collapsing with an empty collection?

## Scala `reduce`

* Given a binary operator, it applies it on collection until down to one element

* Can use the placeholder syntax to have concise expressions

In [3]:
val l = Seq(0, 1, 2, 3, 4, 5)
println ("**** l ****")
println (l)
println ("**** l.reduce{(a, b) => a + b}: ****")
println (l.reduce{(a, b) => a + b })
println ("**** l reduce { _ + _ } ****")
println (l reduce { _ + _ })

val squares = l map { i => i * i }
println ("**** squares ****")
println (squares)

val sumOfSquares = squares reduce { _ + _ }
println ("**** sumOfSquares ****")
println (sumOfSquares)
println ("**** l map { i => i*i } reduce { _ + _ }: ****")
// Error in "_" expanded funciton.
//println (l map { _ * _ } reduce { _ + _ })

**** l ****
List(0, 1, 2, 3, 4, 5)
**** l.reduce{(a, b) => a + b}: ****
15
**** l reduce { _ + _ } ****
15
**** squares ****
List(0, 1, 4, 9, 16, 25)
**** sumOfSquares ****
55
**** l map { i => i*i } reduce { _ + _ }: ****


l: Seq[Int] = List(0, 1, 2, 3, 4, 5)
squares: Seq[Int] = List(0, 1, 4, 9, 16, 25)
sumOfSquares: Int = 55

## Tweaking Our Arbiter with FP (1/2) - original

In [4]:
class MyArb(numPorts: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(w.W))))
        val out = Decoupled(UInt(w.W))
    })
    require (numPorts > 0)
    val inValids = Wire(Vec(numPorts, Bool()))
    val inBits   = Wire(Vec(numPorts, UInt(w.W)))
    val chosenOH = PriorityEncoderOH(inValids)
    for (p <- 0 until numPorts) {
        io.req(p).ready := chosenOH(p) && io.out.fire
        inValids(p) := io.req(p).valid
        inBits(p) := io.req(p).bits
    }
    io.out.valid := inValids.asUInt.orR
    io.out.bits := Mux1H(chosenOH, inBits)
}

defined class MyArb

## Tweaking Our Arbiter with FP (2/2) - with FP + reduce

In [5]:
class MyArb(numPorts: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(w.W))))
        val out = Decoupled(UInt(w.W))
    })
    require (numPorts > 0)
    val inValids = io.req map { _.valid }
//     io.out.valid := VecInit(inValids).asUInt.orR
    io.out.valid := inValids reduce { _ || _ }
    val chosenOH = PriorityEncoderOH(inValids)
    io.out.bits := Mux1H(chosenOH, io.req map { _.bits })
    io.req.zip(chosenOH) foreach { case (i, c) => i.ready := c && io.out.fire}
}

defined class MyArb

## How Do You Reduce on 0 Elements?

* What should `reduce` return when the collection has 0 elements?

* Alternatively, what if we want to collapse a collection into a different type?

## Scala `foldLeft`

* Given initial value and operator, applies _left to right_
  * "Left" is element 0, i.e. in iterable's order
* Can be used to implement `reduce`
* Can return a type different than initial collection

#### Scala Code: foldLeft ####

In [17]:
//val l = Seq(1, 2, 3, 4, 5)
val l = Seq[Int](1, 2, 3, 4, 5)
println ("**** l ****")
println (l)

println ("**** l.foldLeft(0)((totalSoFar, elem) => totalSoFar + elem) ****")
println (l.foldLeft(0)((totalSoFar, elem) => totalSoFar + elem))

println ("**** l.foldLeft(0)(_ + _) ****")
println (l.foldLeft(0)(_ + _))
println ("**** l reduce { _ + _ } ****")
println (l reduce { _ + _ })
println ("**** l.sum ****")
println (l.sum)

def myMax(maxSoFar: Int, x: Int) = if (maxSoFar > x) maxSoFar else x
val maxTheHardWay = l.foldLeft(0)(myMax)
println ("**** l.max ****")
println (l.max)

**** l ****
List(1, 2, 3, 4, 5)
**** l.foldLeft(0)((totalSoFar, elem) => totalSoFar + elem) ****
15
**** l.foldLeft(0)(_ + _) ****
15
**** l reduce { _ + _ } ****
15
**** l.sum ****
15
**** l.max ****
5


l: Seq[Int] = List(1, 2, 3, 4, 5)
defined function myMax
maxTheHardWay: Int = 5

## Visualizing `foldLeft` & `foldRight`

<img src="images/folds.svg" alt="foldLeft & foldRight" style="width:50%; align:left "/>

## Brief Detour: Currying (functions) in Scala

* Multiple argument lists to a function
* We have seen it and used it without talking about it yet
  * e.g. `Seq.fill(4)(0)`
* Can create partially applied functions to pass to FP operation

In [34]:
def sum(a: Int, b: Int) = a + b

def plusX(x: Int)(b: Int) = x + b

val f = plusX(1)_        // give the x = 1, and leave the outher unbound (_).
// val f1 = plusX(_)(1)  // Error

println ("f: " + f)
println ("f(2): " + f(2))  // f(2) => plusX(1)_ => plusX(1)(2) => 1 + 2 = 3 
println("plusX(1)(2): " + plusX(1)(2))  // plusX(1)(2) => 1 + 2 = 3
print ("Seq(0,1,2,3,4) map plusX(10): ")
println (Seq(0,1,2,3,4) map plusX(10)) 
// (0, 1, 2, 3, 4) map plusX(10) => (0, 1, 2, 3, 4) map (x + 10)
// => (0 + 10, 1 + 10, 2 + 10, 3 + 10, 4 + 10) 
// => (10, 11, 12, 13, 14)

f: ammonite.$sess.cmd33$Helper$$Lambda$5766/986641377@38e38244
f(2): 3
plusX(1)(2): 3
Seq(0,1,2,3,4) map plusX(10): List(10, 11, 12, 13, 14)


defined function sum
defined function plusX
f: Int => Int = ammonite.$sess.cmd33$Helper$$Lambda$5766/986641377@38e38244

## Brief Detour: Scala Function Signatures

<img src="images/map.png" alt="map signature" style="width:70%;margin-left:auto;margin-right:auto"/>
<img src="images/foldLeft.png" alt="foldLeft signature" style="width:70%;margin-left:auto;margin-right:auto"/>

* Screenshots from language API docs, will want to peruse for available FP operations
  * Example from [Seq](https://www.scala-lang.org/api/2.13.10/scala/collection/Seq.html)
* Square brackets `[]` indicate parameterized types (generics), and often type inference determines them (e.g. `A`)
* Recognize these operations take in functions (as `op`): (_input arg types_) `=>` _return type_

## `reduce`X vs `fold`Y

* All 6 variants exist (`reduce`, `reduceLeft`, `reduceRight`, `fold`, `foldLeft`, `foldRight`)

* Directions give explicit evaluation order, otherwise unspecified

* In practice, `foldLeft` is often most versatile/appropriate, but brevity of `reduce` makes it tempting
  * Typically use `reduce` to collapse, but `foldLeft` to do it in deliberate order

* Can use `foldRight` and `reduceRight` to effectively do things in reverse (can also use `.reverse` before going from left)

## Redoing Reducer with `reduce`

#### Class Reduce using Loop ####

In [8]:
class Reducer(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))
        val out = Output(UInt(w.W))
    })
    require(n > 0)
    var totalSoFar = io.in(0)
    for (i <- 1 until n)
        totalSoFar = io.in(i) + totalSoFar
    io.out := totalSoFar
//     io.out := io.in.reduce{ _ + _ }
}
//printVerilog(new Reducer(4,2))

defined class Reducer

In [9]:
//printVerilog(new Reducer(4,2))
println (getVerilog (new Reducer(4,2)))

Elaborating design...
Done elaborating.
module Reducer(
  input        clock,
  input        reset,
  input  [1:0] io_in_0,
  input  [1:0] io_in_1,
  input  [1:0] io_in_2,
  input  [1:0] io_in_3,
  output [1:0] io_out
);
  wire [1:0] _T_1 = io_in_1 + io_in_0; // @[cmd7.sc 9:31]
  wire [1:0] _T_3 = io_in_2 + _T_1; // @[cmd7.sc 9:31]
  assign io_out = io_in_3 + _T_3; // @[cmd7.sc 9:31]
endmodule



#### Class Reducer using reduce to replace loop ####

In [35]:
// Class Reducer using reduce
class Reducer(n: Int, w: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Vec(n, UInt(w.W)))
        val out = Output(UInt(w.W))
    })
    require(n > 0)
//    var totalSoFar = io.in(0)
//    for (i <- 1 until n)
//        totalSoFar = io.in(i) + totalSoFar
//    io.out := totalSoFar
    io.out := io.in.reduce{ _ + _ }
}

defined class Reducer

In [36]:
//printVerilog(new Reducer(4,2))
println (getVerilog (new Reducer(4,2)))

Elaborating design...
Done elaborating.
module Reducer(
  input        clock,
  input        reset,
  input  [1:0] io_in_0,
  input  [1:0] io_in_1,
  input  [1:0] io_in_2,
  input  [1:0] io_in_3,
  output [1:0] io_out
);
  wire [1:0] _T_1 = io_in_0 + io_in_1; // @[cmd34.sc 11:31]
  wire [1:0] _T_3 = _T_1 + io_in_2; // @[cmd34.sc 11:31]
  assign io_out = _T_3 + io_in_3; // @[cmd34.sc 11:31]
endmodule



## Redoing DelayN (Pipe) with `foldLeft`

#### Class DelayNCycles Use Recusive ####

In [10]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n >= 0)
    def helper(n: Int, lastConn: Bool): Bool = {
        if (n == 0) lastConn
        else helper(n-1, RegNext(lastConn))
    }
    io.out := helper(n, io.in)
//     io.out := (0 until n).foldLeft(io.in){(lastConn,i) => RegNext(lastConn)}
}
//printVerilog(new DelayNCycles(3))

defined class DelayNCycles

In [11]:
//printVerilog(new DelayNCycles(3))
println (getVerilog (new DelayNCycles(3)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
`endif // RANDOMIZE_REG_INIT
  reg  REG; // @[cmd9.sc 9:33]
  reg  REG_1; // @[cmd9.sc 9:33]
  reg  REG_2; // @[cmd9.sc 9:33]
  assign io_out = REG_2; // @[cmd9.sc 11:12]
  always @(posedge clock) begin
    REG <= io_in; // @[cmd9.sc 9:33]
    REG_1 <= REG; // @[cmd9.sc 9:33]
    REG_2 <= REG_1; // @[cmd9.sc 9:33]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RAN

#### DelayNCycles using foldLeft ####

In [37]:
class DelayNCycles(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val out = Output(Bool())
    })
    require(n >= 0)
    //def helper(n: Int, lastConn: Bool): Bool = {
    //    if (n == 0) lastConn
    //    else helper(n-1, RegNext(lastConn))
    //}
    //io.out := helper(n, io.in)
    io.out := (0 until n).foldLeft(io.in){(lastConn,i) => RegNext(lastConn)}
}

defined class DelayNCycles

In [38]:
println (getVerilog (new DelayNCycles(3)))

Elaborating design...
Done elaborating.
module DelayNCycles(
  input   clock,
  input   reset,
  input   io_in,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
  reg [31:0] _RAND_1;
  reg [31:0] _RAND_2;
`endif // RANDOMIZE_REG_INIT
  reg  REG; // @[cmd36.sc 12:66]
  reg  REG_1; // @[cmd36.sc 12:66]
  reg  REG_2; // @[cmd36.sc 12:66]
  assign io_out = REG_2; // @[cmd36.sc 12:12]
  always @(posedge clock) begin
    REG <= io_in; // @[cmd36.sc 12:66]
    REG_1 <= REG; // @[cmd36.sc 12:66]
    REG_2 <= REG_1; // @[cmd36.sc 12:66]
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin

## Scala `zipWithIndex`

* Sometimes want to have access to index when performing FP op
  * Analogous to `enumerate` in Python

In [47]:
// Print the list and index.
val l = Seq(5,6,7,8)
for (x <- l) {
    println(x)
}
for (i <- 0 until l.size) {
    println (l(i), i)
}

5
6
7
8
(5,0)
(6,1)
(7,2)
(8,3)


l: Seq[Int] = List(5, 6, 7, 8)

In [49]:
// Use zip to print the list and index.
val l = Seq(5,6,7,8)
print ("l.zip(0 until l.size): ")
println (l.zip(0 until l.size))
print ("l.zip(0 until l.size): ")
println (l.zip(0 until l.size))
print ("l.zip: ")
println (l.zipWithIndex)
print ("l.zipWithIndex.map{ t => t._1 * t._2 }: ")
println (l.zipWithIndex.map{ t => t._1 * t._2 })
print ("l.zipWithIndex.map{ case (x, i) => x * i }: ")
println (l.zipWithIndex.map{ case (x, i) => x * i })

l.zip(0 until l.size): List((5,0), (6,1), (7,2), (8,3))
l.zip(0 until l.size): List((5,0), (6,1), (7,2), (8,3))
l.zip: List((5,0), (6,1), (7,2), (8,3))
l.zipWithIndex.map{ t => t._1 * t._2 }: List(0, 6, 14, 24)
l.zipWithIndex.map{ case (x, i) => x * i }: List(0, 6, 14, 24)


l: Seq[Int] = List(5, 6, 7, 8)

## `zipWithIndex` Diagram

<p>
<img src="images/zipWithIndex.svg" alt="zipWithIndex" style= "width:50%; align:left" />

## One-Hot Priority Encoder (with muxes) Redone with FP

In [13]:
class MyPriEncodeOH(n: Int) extends Module {
    val io = IO(new Bundle {
        val in  = Input(UInt(n.W))
        val out = Output(UInt())
    })
    require (n > 0)
    def withMuxes(index: Int): UInt = {
        if (index < n) Mux(io.in(index), (1 << index).U, withMuxes(index+1))
        else 0.U
    }
    io.out := withMuxes(0)
//     io.out := io.in.asBools.zipWithIndex.reverse.foldLeft(0.U) {
//         case (soFar, (b, index)) => Mux(b, (1 << index).U, soFar)
// //         case ((b, index), soFar) => Mux(b, (1 << index).U, soFar)
//     }
//     io.out := PriorityEncoderOH(io.in)    // Standard Library
    printf("%b -> %b\n", io.in, io.out)
}

// printVerilog(new MyPriEncodeOH(3))
test(new MyPriEncodeOH(3)) { c =>
    for (i <- 0 until 8) {
        c.io.in.poke(i.U)
        c.clock.step()
    }
}

Elaborating design...
Done elaborating.
  0 ->   0
  1 ->   1
 10 ->  10
 11 ->   1
100 -> 100
101 ->   1
110 ->  10
111 ->   1
  0 ->   0
test MyPriEncodeOH Success: 0 tests passed in 10 cycles in 0.059343 seconds 168.51 Hz


defined class MyPriEncodeOH

## Redoing Crossbar with FP (1/4) - IO decs

In [14]:
class Message(numOuts: Int, length: Int) extends Bundle {
    val addr = UInt(log2Ceil(numOuts).W)
    val data = UInt(length.W)
}

class XBarIO(numIns: Int, numOuts: Int, length: Int) extends Bundle {
    val in  = Vec(numIns, Flipped(Decoupled(new Message(numOuts, length))))
    val out = Vec(numOuts, Decoupled(new Message(numOuts, length)))
}

defined class Message
defined class XBarIO

## Redoing Crossbar with FP (2/4) - inner loops only

In [15]:
class XBar(numIns: Int, numOuts: Int, length: Int) extends Module {
    val io = IO(new XBarIO(numIns, numOuts, length))
    val arbs = Seq.fill(numOuts)(Module(new RRArbiter(new Message(numOuts, length), numIns)))
    for (ip <- 0 until numIns) {
        val inReadys = Wire(Vec(numOuts, Bool()))
        for (op <- 0 until numOuts) {
            inReadys(op) := arbs(op).io.in(ip).ready
        }
        io.in(ip).ready := inReadys.asUInt.orR
//         io.in(ip).ready := arbs.map{ _.io.in(ip).ready }.reduce{ _ || _ }
    }
    for (op <- 0 until numOuts) {
        for (ip <- 0 until numIns) {
            arbs(op).io.in(ip).bits <> io.in(ip).bits
            arbs(op).io.in(ip).valid := io.in(ip).valid && (io.in(ip).bits.addr === op.U)
        }
//         arbs(op).io.in.zip(io.in).foreach { case (arbIn, ioIn) =>
//             arbIn.bits <> ioIn.bits
//             arbIn.valid := ioIn.valid && (ioIn.bits.addr === op.U)
//         }
        io.out(op) <> arbs(op).io.out
    }
    for (op <- 0 until numOuts) {
        printf(" %d -> %d (%b)", io.out(op).bits.data, op.U, io.out(op).valid)
    }
    printf("\n")
}

defined class XBar

## Redoing Crossbar with FP (3/4) - all loops

In [16]:
class XBar(numIns: Int, numOuts: Int, length: Int) extends Module {
    val io = IO(new XBarIO(numIns, numOuts, length))
    val arbs = Seq.fill(numOuts)(Module(new RRArbiter(new Message(numOuts, length), numIns)))
    for (ip <- 0 until numIns) {
        io.in(ip).ready := arbs.map{ _.io.in(ip).ready }.reduce{ _ || _ }
    }
//     io.in.zipWithIndex.foreach { case (in, ip) =>
//         in.ready := arbs.map{ _.io.in(ip).ready }.reduce{ _ || _ }
//     }
    for (op <- 0 until numOuts) {
        arbs(op).io.in.zip(io.in).foreach { case (arbIn, ioIn) =>
            arbIn.bits <> ioIn.bits
            arbIn.valid := ioIn.valid && (ioIn.bits.addr === op.U)
        }
        io.out(op) <> arbs(op).io.out
    }
//     io.out.zip(arbs).zipWithIndex.foreach { case ((ioOut, arbOut), op) =>
//         arbOut.io.in.zip(io.in).foreach { case (arbIn, ioIn) =>
//             arbIn.bits <> ioIn.bits
//             arbIn.valid := ioIn.valid && (ioIn.bits.addr === op.U)
//         }
//         ioOut <> arbOut.io.out
//     }
    for (op <- 0 until numOuts) {
        printf(" %d -> %d (%b)", io.out(op).bits.data, op.U, io.out(op).valid)
    }
//     io.out.zipWithIndex.foreach{
//         case (outP, op) => printf(" %d -> %d (%b)", outP.bits.data, op.U, outP.valid)
//     }
    printf("\n")
}

defined class XBar

In [16]:
//
//println (getVerilog (new XBar(1, 1, 8)))

## Redoing Crossbar with FP (4/4) - Tests

In [16]:
/*
val numIns = 4
val numOuts = 2
test(new XBar(numIns,numOuts,8)) { c =>
    for (ip <- 0 until numIns) {
        c.io.in(ip).valid.poke(true.B)
        c.io.in(ip).bits.data.poke(ip.U)
        c.io.in(ip).bits.addr.poke((ip % numOuts).U)
    }
    for (op <- 0 until numOuts) {
        c.io.out(op).ready.poke(true.B)
    }
    for (cycle <- 0 until 4) {
        c.clock.step(0)
    }
    c.clock.step(4)
}
*/

## Only Use FP When it is an Improvement!

* FP used well...
  * Leverages FP operation to execute commmon pattern
  * Improves readability and simplifies code

* FP used over-eagerly...
  * Harder to read/understand
  * Brittle

* Consider...
  * Would a simple for loop or even recursion be more clear?
  * Limit self to 2-3 FP operations per line at most
  * Multiple lines for the function literal?
    * Maybe pull into a named helper function or fall back to _for_